In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- PHẦN 1: LOAD VÀ KẾT HỢP DỮ LIỆU ---

# 1. Load dữ liệu
df_layoff = pd.read_csv('../data/processed/tech_layoffs_clean.csv')
df_macro = pd.read_csv('../data/raw/macro_economics_data.csv')

# 2. Xử lý ngày tháng
df_layoff['date'] = pd.to_datetime(df_layoff['date'])

# Bên file macro, cột ngày đang bị đặt tên là 'Unnamed: 0' hoặc 'date'
if 'Unnamed: 0' in df_macro.columns:
    df_macro = df_macro.rename(columns={'Unnamed: 0': 'date'})
df_macro['date'] = pd.to_datetime(df_macro['date'])

# 3. Gom nhóm Layoff theo tháng (Resample)
# Vì dữ liệu Macro là theo tháng, nên ta phải tính TỔNG Layoff trong tháng đó
df_layoff_monthly = df_layoff.set_index('date').resample('MS')['laid_off_count'].sum().reset_index()

# 4. Merge (Kết hợp) 2 bảng lại
df_merged = pd.merge(df_layoff_monthly, df_macro, on='date', how='inner')

print("✅ Đã merge thành công! Dữ liệu sẵn sàng:")
print(df_merged.head())

# --- PHẦN 2: VẼ BIỂU ĐỒ TÁC ĐỘNG (DUAL AXIS) ---

fig, ax1 = plt.subplots(figsize=(12, 6))

# Trục 1 (Trái): Số lượng sa thải (Cột màu đỏ)
color_1 = 'tab:red'
ax1.set_xlabel('Date (Year-Month)', fontsize=12)
ax1.set_ylabel('Tech Layoffs (People)', color=color_1, fontsize=12)
ax1.bar(df_merged['date'], df_merged['laid_off_count'], color=color_1, alpha=0.6, width=20, label='Layoffs')
ax1.tick_params(axis='y', labelcolor=color_1)

# Trục 2 (Phải): Lãi suất Fed (Đường màu xanh)
ax2 = ax1.twinx()  
color_2 = 'tab:blue'
ax2.set_ylabel('Fed Interest Rate (%)', color=color_2, fontsize=12)  
ax2.plot(df_merged['date'], df_merged['Interest_Rate'], color=color_2, linewidth=3, linestyle='-', marker='o', markersize=4, label='Interest Rate')
ax2.tick_params(axis='y', labelcolor=color_2)

# Trang trí
plt.title('The End of "Cheap Money": Interest Rates vs. Tech Layoffs', fontsize=16, fontweight='bold')
plt.grid(True, linestyle='--', alpha=0.3)
fig.tight_layout()  
plt.show()

# --- PHẦN 3: MA TRẬN TƯƠNG QUAN (CORRELATION MATRIX) ---

# Chọn các chỉ số quan trọng để soi
cols_to_analyze = ['laid_off_count', 'Interest_Rate', 'Inflation_CPI', 'Unemployment_Rate', 'GDP']
corr_matrix = df_merged[cols_to_analyze].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Matrix: Macro Indicators vs. Layoffs')
plt.show()